In [ ]:
import atlite
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import xarray as xr
from rasterio.plot import show
import rasterio
import rioxarray as rxr
from atlite.gis import shape_availability, ExclusionContainer
import yaml
import functools
import numpy as np
import hvplot.xarray

In [ ]:
with open("land-eligibility.yaml", "r") as f:
    C = yaml.safe_load(f)

In [ ]:
PATH = "../../../playgrounds/pr/pypsa-eur/"
NATURA = PATH + "resources/natura.tiff"
COUNTRIES = PATH + "resources/country_shapes.geojson"
REGIONS = PATH + "resources/regions_onshore_elec_s_181.geojson"
CORINE = PATH + "data/bundle/corine/g250_clc06_V18_5.tif"
GEBCO = PATH + "data/bundle/GEBCO_2014_2D.nc"
OFFREGIONS = PATH + "resources/regions_offshore_elec_s_181.geojson"
EEZ = PATH + "resources/offshore_shapes.geojson"
RES = 250
CRS = 3035
SIZES = [20, 40, 60]
TECHS = ["solar", "offwind-floating"]

In [ ]:
countries = gpd.read_file(COUNTRIES).set_index("name")
regions = gpd.read_file(REGIONS).set_index("name")
eez = gpd.read_file(EEZ).set_index("name")
offregions = gpd.read_file(OFFREGIONS).set_index("name")

In [ ]:
def eligibility(shapes, config, crs, res):

    excluder = atlite.ExclusionContainer(crs=crs, res=res)

    shapes = shapes.geometry.to_crs(excluder.crs)

    if config["natura"]:
        excluder.add_raster(NATURA, nodata=0, allow_no_overlap=True)

    corine = config.get("corine", {})

    if "grid_codes" in corine:
        codes = corine["grid_codes"]
        excluder.add_raster(CORINE, codes=codes, invert=True, crs=3035)
    if corine.get("distance", 0.0) > 0.0:
        codes = corine["distance_grid_codes"]
        buffer = corine["distance"]
        excluder.add_raster(CORINE, codes=codes, buffer=buffer, crs=3035)

    if "max_depth" in config:
        func = functools.partial(np.greater, -config["max_depth"])
        excluder.add_raster(GEBCO, codes=func, crs=4236, nodata=-1000)

    if "min_shore_distance" in config:
        buffer = config["min_shore_distance"]
        excluder.add_geometry(COUNTRIES, buffer=buffer)

    if "max_shore_distance" in config:
        buffer = config["max_shore_distance"]
        excluder.add_geometry(COUNTRIES, buffer=buffer, invert=True)

    masked, transform = shape_availability(shapes, excluder)

    return masked, transform

In [ ]:
def plot(masked, transform, countries, eez, size=60, fn=None):

    fig, ax = plt.subplots(figsize=(size, size))

    countries = countries.geometry.to_crs(CRS)
    eez = eez.geometry.to_crs(CRS)

    show(masked, transform=transform, cmap="Greens", ax=ax)

    contour_kws = dict(edgecolor="k", linewidth=2, color="None")
    countries.plot(ax=ax, **contour_kws)
    if "off" in tech:
        eez.plot(ax=ax, **contour_kws)

    plt.box(False)
    plt.axis("off")
    if fn is not None:
        plt.savefig(fn, bbox_inches="tight")

In [ ]:
def save(masked, transform, fn, crs):

    ds = rasterio.open(
        fn,
        "w",
        driver="GTiff",
        height=masked.shape[0],
        width=masked.shape[1],
        count=1,
        dtype=str(masked.dtype),
        crs=CRS,
        transform=transform,
    )

    ds.write(masked, 1)
    ds.close()

In [ ]:
for tech in TECHS:

    print(tech)

    config = C[tech]
    shapes = offregions if "off" in tech else regions
    masked, transform = eligibility(shapes, config, CRS, RES)

    for size in SIZES:
        print(f"- plot {size}")
        fn = f"../results/graphics/eligibility-{tech}-{RES}-{size}.pdf"
        plot(masked, transform, countries, eez, size, fn)

    print("- save raster")
    fn = f"../results/data/eligibility-{tech}-{RES}.tif"
    save(masked, transform, fn, CRS)

In [ ]:
if "ds" in globals():
    ds.close()
ds = rxr.open_rasterio("../results/data/eligibility-onwind-1000.tif")

In [ ]:
ds.sel(band=1).hvplot(
    x="x",
    y="y",
    tiles="OSM",
    geo=True,
    crs=CRS,
    alpha=0.5,
    frame_height=800,
    colorbar=False,
    legend=False,
    hover=False,
    title="Available Land",
    cmap="Greens",
    # dynamic=False,
    # rasterize=True,
    # project=True,
).opts(active_tools=["pan", "wheel_zoom"])